<a href="https://colab.research.google.com/github/szostaklab/aminoacylation/blob/main/n40_structure_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title IMPORTS
#Install+Import Packages
!pip install mechanize
import random
import re
import mechanize
import numpy as np
from random import *
import matplotlib.pyplot as plt
!pip install Bio
from Bio import SeqIO
from Bio.Seq import Seq
from collections import Counter 
from Bio.Cluster import kcluster
import statistics
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler
import difflib
import seaborn as sns; sns.set_theme()
from collections import defaultdict
import collections
from mpl_toolkits.mplot3d import Axes3D
from pylab import *
from collections import OrderedDict
import matplotlib.pyplot as plt
#!pip install ipython-autotime

In [11]:
#@title Upload Data and Define Parameters
aa_sequences = open('Str_S1_L001_R1_001.fastq').readlines()
rna_only_sequences = open('Str-NaOH_S2_L001_R1_001.fastq').readlines()

#Take the reverse complement? Enter 'yes' or 'no'
rc = 'no'

#Quality Cutoff - what fraction of positions should have <1% error probability?
quality_cutoff = 0.9

#Position motif1 starts at
motif_start = 1
#Motif1 define
motif = 'AAA'

#Position motif2 starts at
motif2_start = 42
#Motif2 define
motif2 = 'AAA'

In [12]:
#@title STATISTICS (note: this takes a few mins per round)
#Remove sequences which are present in the RNA only list and keep only correct constant region 
print('Number of seqs in AA: '+str(len(aa_sequences)/4)) 

aa_minus_rna_seqs = []
for seq in range(len(aa_sequences)):
  if (seq-1)%4==0:
      if aa_sequences[seq][motif_start-1:(motif_start-1)+len(motif)] == motif and aa_sequences[seq][motif2_start-1:(motif2_start-1)+len(motif2)] == motif2:
        aa_minus_rna_seqs+=(aa_sequences[seq-1:seq+3])
quality = aa_minus_rna_seqs[3::4]
seqs = aa_minus_rna_seqs[1::4]
print('Number of seqs with correct constant region in AA but not RNA: '+str(len(seqs))) 

#Remove duplicates:
unique_seqs = []
unique_qualities = []
indicies_of_unique_seqs = list(np.unique(seqs, return_index=True)[1])
for index in sorted(np.unique(seqs, return_index = True)[1])[:-1]:
  unique_qualities.append(quality[index])
  unique_seqs.append(seqs[index])
print('Number of unique seqs: '+str(len(unique_seqs))) 

#Counts of all seqs
seq_dict = Counter(seqs)
seq_dict_list = [(k, v) for k, v in seq_dict.items()]
sorted_seq_dict = sorted(seq_dict_list, key = lambda x: x[1], reverse=True)
sorted_seq_dict_inve = sorted(seq_dict_list, key = lambda x: x[1])

#Take RC if desired
pre_trim = []
if rc == 'yes':
  rc_seqs = []
  for sequence in unique_seqs:
      rc_seqs.append(str(Seq(sequence).reverse_complement()).strip('\n'))
  pre_trim = rc_seqs
else:
  for sequence in unique_seqs:
      pre_trim.append(str(Seq(sequence)).strip('\n'))
if rc == 'yes':
  print('Number of seqs after rc: '+str(len(pre_trim))) 

#Trim
trimmed = []
for seq in pre_trim:
  trimmed.append(seq[(motif_start-1):(motif2_start-1)])

print('Number of seqs after trimming: '+str(len(trimmed))) 

#Filter by quality
high_quality = []
acceptable_quals_1_percent_error = [5,6,7,8,9,':',';', '<', '=', '>', '?', '@', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K']
for seq in range(len(unique_qualities)):
  overlap=0
  for char in list(set(unique_qualities[seq]).intersection(acceptable_quals_1_percent_error)):
        overlap+=unique_qualities[seq].count(char)
  if overlap/len(unique_qualities[1])>quality_cutoff:
        high_quality.append(trimmed[seq].strip("\n"))
print('Number of high quality seqs: '+str(len(high_quality))) 

Number of seqs in AA: 3006527.0
Number of seqs with correct constant region in AA but not RNA: 229456
Number of unique seqs: 228862
Number of seqs after trimming: 228862
Number of high quality seqs: 223371


In [13]:
#@title TOP SEQUENCES (ensure high quality)
#Top Sequences 
print('Most Abundant Sequences, Counts')
top_seqs = []

for seq in sorted_seq_dict[:30]:
  for high_qual in high_quality:
    if high_qual in seq[0]:
      top_seqs.append(high_qual[motif_start+2:motif2_start-1])
      break

for top in top_seqs:
  counts=0
  for seq in seqs:
    if top in seq:
      counts+=1
  print(top, counts)

Most Abundant Sequences, Counts
ATGCAACTGGACAATCAGAAAGAGATTGCCGAGATGCA 77
AGTCAGATATGGACCTTGCTGCTAAAGGTCTAGGAGCT 52
TTGTTCCAAGTATCGGCAACAGCTTTATCAATACCATG 45
TTACGTGCAGAAGGAGTGATGTAATGTCTAAAGGTAAA 110
TTTAGGGTCGGCATCAAAAGCAATATCAGCACCAACAG 41
ATTACGTGCAGAAGGAGTGATGTAATGTCTAAAGGTAA 110
TGAAGACGGCCATTAGCTGTACCATACTCAGGCACACA 37
CGCCTCTAATCGGTCGTCAGCCAACGTGAGAGTGTCAA 63
ACCTACCGCGCTTCGCTTGGTCAACCCCTCAGCGGCAA 34
CTTTGTTACTCGTCAGAAAATCGAAATCATCTTCGGTT 29
ATTACGTGCAGAAGGAGTGATGTAATGTCTAAAGGTAA 110
CGCCTCTAATCGGTCGTCAGCCAACGTGAGAGTGTCAA 63
TATCCGAAAGTGTTAACTTCTGCGTCATGGAAGCGATA 27
CACTGACGTTCTTACTGACGCAGAAGAAAACGTGCGTC 27
TAAATTTCTAGGTGGTATGATGGTTAGAAAATCACAGT 2
GCGGTCATGGCTGTTCTATCACAGCACTGGCGACTGGC 2
GGTTATGTCGACGACGTAGATTTGGTATCAGGCTGTGT 2
GGTCATGGTGTATATTTACTGTGTAACAATGGTCTGTA 2
GTTAAGTGGAGGCCGCATACCAAAGCCGGGGGTGGAGT 2
GTTTATATTGGGTGCAAGCTGTTTGTGTCAAATGTGAA 2
CGGGGAATGGTTCCATATAGTAGGTCACTTGGTGGATT 2
AGGACTGGGTGGGAATGCCTTGCAAGCGTCAACGAAGC 2
GTGGAGTTGTCAGGTCACAGGGGCGCTATATGTTGAGG 2
GTTTTATA

In [14]:
#@title Function to get secondary structure on RNA Fold

def get_energy_ss(seq):
  url = "http://rna.tbi.univie.ac.at//cgi-bin/RNAWebSuite/RNAfold.cgi"
  br = mechanize.Browser()
  br.set_handle_robots(False) # ignore robots
  br.open(url)
  br.select_form(name="form")
  br["SCREEN"] = seq
  res = br.submit()
  content = res.read()

  energy = int(re.sub("[^0-9]", "", str(content).split('thermodynamic ensemble is')[1].split('kcal/mol')[0]))
  structure = str(content).split('STRUCTURE=')[1].split('target')[0].split('"')[0]
  return energy, structure

In [18]:
#@title Get the SS of the top sequences
ss_of_top_seqs=[]
for seq in top_seqs:
  if top_seqs.index(seq)%2==0:
    print(top_seqs.index(seq))
  ss_of_top_seqs.append(get_energy_ss(seq[motif_start+2:motif2_start-1])[1])

0
2
4
6
8
12
14
16
18
20
22
24
26
28


In [19]:
#@title Cluster by secondary structure
k = 4
matrix = np.asarray([np.fromstring(str(s), dtype=np.uint8) for s in ss_of_top_seqs]);
kmeans = KMeans(init="random", n_clusters=k,n_init=10,max_iter=300,random_state=42)
kmeans.fit(matrix)
labels = kmeans.labels_[:]

all_clusters = []
for cluster_num in range(0, k):
  seqs_in_cluster = []
  for seq in range(len(labels)):
    if labels[seq] == cluster_num:
      seqs_in_cluster.append(ss_of_top_seqs[seq])
  all_clusters.append(seqs_in_cluster)
all_clusters

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  This is separate from the ipykernel package so we can avoid doing imports until


[['...........(((....)))..............',
  '..(((((.....)))))..................',
  '......((..((((.((......))..))))))..',
  '...............(((((((......)))))))',
  '.............(((((.......))))).....',
  '.............((((((..........))))))',
  '.........((((...((......))..))))...',
  '........(((((..(((.....)))..)))))..',
  '...(((((....)))))..................'],
 ['..((((..........))))...............',
  '(((((((..........))).))))..........',
  '(((((((..........))).))))..........',
  '.(((((((.((......)).)))))))........',
  '((((((...))))))(((.....))).........',
  '(((((((......)))))))...............',
  '.(((.(((..........))))))...........',
  '(((.(((((..........))))).))).......'],
 ['(((.....(((.....))).....)))........',
  '(((.....(((.....))).....)))........',
  '((((..(((((......)))))..)))).......',
  '.((...(((......)))....))((((...))))'],
 ['....(((.(((((.......)))))))).......',
  '......((((((((.......))))))))......',
  '..(((..((................))))).....',
  '((((((.(((